In [1]:
from prevo.record import RecordBase, RecordingBase, SensorBase
from prevo.csv import RecordingToCsv
from prevo.misc import DummyPressureSensor, DummyTemperatureSensor
import oclock

# Define sensors

In [2]:
class Sensor(SensorBase):
    """Provide methods common to all sensors"""
    
    def  _read(self):
        # Automatically add time and time uncertainty in data dict
        with oclock.measure_time() as data:
            measurement = self.device.read()
            data['values'] = tuple(measurement.values())
        return data

In [3]:
class PressureSensor(Sensor):
    
    name = 'P'
    
    # This will be the names of columns in the CSV file
    # (as can be seen below, time columns are also added automatically)
    data_names = 'P1', 'P2', 'P3'
    
    # data_types is useful only when using live graphs
    data_types = 'P (Pa)', 'P (Pa)', 'P (mbar)'
    
    def __init__(self):
        self.device = DummyPressureSensor()


class TemperatureSensor(Sensor):
    
    name = 'T'
    data_names = 'T1', 'T2'
    data_types = 'T (°C)', 'T (°C)', 'P (mbar)'
    
    def __init__(self):
        self.device = DummyTemperatureSensor()

In [4]:
with PressureSensor() as sensor:
    data = sensor.read()
data

{'values': (3170.577111169146, 2339.897274189945, 17.190375310469857),
 'time (unix)': 1688746887.72925,
 'dt (s)': 8.437491487711668e-06}

In [5]:
Sensors = {Sensor.name: Sensor for Sensor in (PressureSensor, TemperatureSensor)}
Sensors

{'P': __main__.PressureSensor, 'T': __main__.TemperatureSensor}

# Define recordings

In [6]:
class Recording(RecordingToCsv, RecordingBase):
    """Recording class that defines how to read and handle data from sensors."""

    def __init__(self, name):
        
        Sensor = Sensors[name]
        column_names = ('time (unix)', 'dt (s)') + Sensor.data_names
        filename = 'Pressure.tsv' if name == 'P' else 'Temperature.tsv'

        RecordingBase.__init__(self, Sensor=Sensors[name], dt=2)
        RecordingToCsv.__init__(self, path='data', filename=filename, column_names=column_names)
        
recordings = {name: Recording(name) for name in ('P', 'T')}

# Start recording

In [7]:
class Record(RecordBase):
    """For simple cases no subclassing is necessary"""
    pass

record = Record(recordings, path='data')
record.start()

Recording started in folder /Users/olivier.vincent/Python-OV/prevo/examples/data


Type command (help: ?):  ?


OBJECTS ========================================================================
--- P [<__main__.Recording object at 0x11c3a5e10>]
--- T [<__main__.Recording object at 0x11c3a40d0>]
COMMANDS =======================================================================
--- Properties
--- Events
        g, graph -- graph
--- Exit
        q, Q, quit


Type command (help: ?):  q


CLI stopped
Recording Stopped
